Do you need to extract text from different files such as pdfs and Word files? This quick tutorial shows how sort files by type, and then extract text from PDF files. I downloaded fake resumes in pdf format from Overleaf to demonstrate how this code works. I am not going to cover how to extract text from Word documents. You can download ***docxpy*** Python package and use to to extract text from Word files. Feel free to contact me at anna@sakura-ai.com if you have any questions or need help parsing documents. 

The main challenge in extracting text from PDF files is that they have different formats: 

* PDF files are either 8-bit binary files or 7-bit ASCII text files (using ASCII-85 encoding).
* Every line in a PDF can contain up to 255 characters.
* Every line ends with a carriage return, a line feed or a carriage return followed by a line feed (depending upon the application or platform used to create the PDF file).
* PDF is case sensitive.
* The file format is completely independent from the platform that it is viewed or created on. Files can be moved back and forth between Macs, Windows system, Linux systems,... When FTP-ing a PDF file, it does make sense to compress it, to avoid data corruption by some outdated web system that the file needs to go through.
* Scanned PDFs are stored as images

You can learn more about PDF files here: https://www.prepressure.com/pdf/basics/fileformat

My solution to this problem is to convert all PDF files into one format - images using ***pdf2image*** Python package and then use optical character recognition (OCR) Python package to extract text from images. 

First, import all packages. You need ***pdf2image*** to convert pdfs to ppm image files. We will do some path manipulaton to join and rename text files, so we import ***os*** and ***sys*** packages. Next part is a part of calling a library ***PIL*** and importing ***Image*** with ***pytesseract***. You can see full ***pytesseract*** import and usage instructions here: https://pypi.org/project/pytesseract/

In [34]:
#Import all packages

import pdf2image
import os, sys
try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract

#import docxpy  - you can use this package when you are extracting text from word files

Initialize the path to your documents and the counter to use later in pdf extract function to count your documents in the folder.

In [35]:
#initialize the path to your documents
PATH = '/home/anna/Desktop/NLPmain/NLP-class/'

#initialize the counter that you will use later in your pdf extraction function
i = 1

After all text is extracted from the image files, we want these image files erased to avoid flooding the folder that contains the documents with image files. In addition, in the Apple macOS operating system, . DS_Store is a file that stores custom attributes of its containing folder, such as the position of icons or the choice of a background image. It is created when PDF files are convertied to ppm image files. The name is an abbreviation of Desktop Services Store, reflecting its purpose.Since we are going to sort files by extension, these files can prevent our code from running, so we are just going to erase them as well.
Below is delete_ppms function that cleans up all unnecessary files from the document folder - it uses ***os*** Python package, which provides a portable way of using operating system dependent functionality. You can see more documenation on ***os*** package here: https://docs.python.org/3/library/os.html 

In [36]:
def delete_ppms():
  for file in os.listdir(PATH):
    if '.ppm' in file or '.DS_Store' in file:
      try:
          os.remove(PATH + file)
      except FileNotFoundError:
          pass

Now we need to sort files by type. We will use file extension to determine its type. Since I only have Word files and PDF files in my folder, I will only initialize two lists for each extenstion type where I will store the names of the files.
In this for loop, I fist join the path and file names together to ensure their accessibility. Then I split each file name into its name and extension, which enables me to append the file names into two different lists based on their extension type. 

In [37]:
# initialize lists for each document type
pdf_files = []
docx_files = []

# append document names into the lists by their extension type
for f in os.listdir(PATH):
  full_name = os.path.join(PATH, f) #is it path.join or str.join
  if os.path.isfile(full_name):
    name = os.path.basename(f)
    filename, ext = os.path.splitext(name)
    if ext == '.pdf':
      pdf_files.append(name)
    elif ext == ('.docx'):
      docx_files.append(name)

Now we can finally extract text from our documents. Below is ***pdf_extract*** function. First, it is printing from each file it is currently exracting the text. Depending on the size of the document, text extraction can take some time. This print statement will help you see which file is beng extracted at the moment.

Since this function is going to be used in a for loop for each file, it is important to use ***delete ppms*** function each time before extraction to clean up image files from each document page to prevent text from two different documents to be written into the same text file. 
 
Then all files are converted to images, sorted and the images are renamed with the same name plus the document index plus the page index with ppm extension. The files will be sorted to keep order in which the files are renamed to keep each page number from the document in order.

Next, a text file is created for each image. I chose to name text files 'result' with a number extension for each document index. This naming procedure would help  me quickly check if all files were extracted and helps me to combine all pages from the same document into the same text file. You can play with ***os*** package to rename text files to your liking. 

Then all ppm image files are sorted again. The lambda function is created to sort the files based on their names and page numbers.

And finally text is written from images into text files created earlier. 

In [38]:
def pdf_extract(file, i):
  print("extracting from file:", file)
  delete_ppms()
  images = pdf2image.convert_from_path(PATH + file, output_folder=PATH)
  j = 0
  for file in sorted (os.listdir(PATH)):
      if '.ppm' in file and 'image' not in file:
        os.rename(PATH + file, PATH + 'image' + str(i) + '-' + str(j) + '.ppm')
        j += 1
  j = 0
  f = open(PATH +'result{}.txt'.format(i), 'w')
  files = [f for f in os.listdir(PATH) if '.ppm' in f]

  for file in sorted(files, key=lambda x: int(x[x.index('-') + 1: x.index('.')])):
      temp = pytesseract.image_to_string(Image.open(PATH + file))
      f.write(temp)
  f.close()

At last we can run our ***pdf_extract*** function on all pdf files appeneded earlier.

In [33]:
for i in range(len(pdf_files)):
  pdf_file = pdf_files[i]
  pdf_extract(pdf_file, i)


extracting from file: sixtysecondscv.pdf
extracting from file: jakes-resume.pdf


Now if you go to your folder, you should see two text files named ***result0.txt*** and ***result1.txt*** for each resume. 

Thank you for reading my tutorial! Please leave comments below with suggestions on how to edit and format this tutorial. 